<a href="https://colab.research.google.com/github/fellis-cp/Temu-Kembali-Informasi/blob/main/Tokenize_Indo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install PySastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 3.9 MB/s eta 0:00:00


In [33]:
#Cleaning tanda baca dan tokenisasi unigram
import re
import numpy as np
import nltk
import pandas as pd
#download database corpus
nltk.download('punkt')
nltk.download('stopwords')
#menyimpan kata yang terambil dan simpan dalam file word_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer
#import stopwords
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
document1 = "I went to the campus to study"
document2 = "Mother went shopping to the market"
document3 = "Father works at the office to work"
corpus_doc=[document1,document2,document3]
sentence_corpus = pd.DataFrame(corpus_doc)
print(sentence_corpus)

                                    0
0       I went to the campus to study
1  Mother went shopping to the market
2  Father works at the office to work


In [35]:
#Step 1 Cleaning text

#ubah seluruh huruf menjadi huruf kecil
doc = sentence_corpus[0].str.lower()

#Remove Puncutuation
clean_spcl = re.compile('[!@#$%^&*()_+-<>?/.:;"{}’[]|\]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text
# Buat kolom tambahan untuk data description yang telah diremovepunctuation
sentence_corpus['clean_punct'] = doc.apply(clean_punct)
Cleaning_Doc=sentence_corpus['clean_punct']
print(Cleaning_Doc)


0         i went to the campus to study
1    mother went shopping to the market
2    father works at the office to work
Name: clean_punct, dtype: object


In [36]:
#Step normalize Documents
doc=Cleaning_Doc
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")
sentence_corpus['clean_double_ws'] = doc.apply(_normalize_whitespace)
sentence_corpus['clean_double_ws']


0         i went to the campus to study
1    mother went shopping to the market
2    father works at the office to work
Name: clean_double_ws, dtype: object

In [37]:
def stemmingEnglishList(documents):
  stemmer = PorterStemmer()
  stemmed_documents = [stemmer.stem(doc) for doc in documents]
  return stemmed_documents

stemmed_documents = stemmingEnglishList(sentence_corpus['clean_double_ws'])
stemmed_documents
df = pd.DataFrame(stemmed_documents)
sentence_stem = df.head()
sentence_stem


,0
0,i went to the campus to studi
1,mother went shopping to the market
2,father works at the office to work


In [38]:
doc = df[0]
doc

0         i went to the campus to studi
1    mother went shopping to the market
2    father works at the office to work
Name: 0, dtype: object

In [39]:
stopwords = set(stopwords.words("english"))
ST=stopwords
print(ST)

{"haven't", 'or', "aren't", "needn't", 'after', 've', 'some', 'himself', 'shouldn', 'their', 'ourselves', "wouldn't", 'him', 'off', 'until', 'has', 'down', 'she', 'where', "shouldn't", 'd', 'will', 'so', "won't", 'me', 'are', 'such', 'in', "weren't", 'through', 'was', 'they', 'an', "wasn't", 're', 'than', 'but', 'am', 'o', 'with', 'for', 'during', 'nor', 'wasn', 'our', 'very', 's', 'hers', 'its', 'any', 'this', 'that', 'mustn', 'did', 'll', 'we', 'won', 'were', 'ma', "mustn't", 'can', 'his', 'theirs', 'doesn', 'yourselves', 'under', 'again', 'here', "doesn't", 'had', 'wouldn', 'them', 'yourself', 'on', 'ours', "you're", 'having', 'then', 'does', 'isn', 'he', 'couldn', 'itself', 'have', 'no', 'while', "it's", 'by', 'be', 'whom', 'do', 'aren', 'hadn', 'hasn', 'when', "hadn't", 'about', "hasn't", 'once', 'over', 'between', 'is', 'what', 'before', 'few', 'y', 'ain', 'mightn', 'too', "that'll", 'those', 'now', 'shan', 'needn', 'themselves', "didn't", 'against', "you'd", 'to', 'how', 'should

In [21]:
def clean_stopwords(text):
    text = ' '.join(word for word in text.split() if word not in ST)
    return text
df['clean_sw'] = doc.apply(clean_stopwords)
doc_clean=df['clean_sw']
print(doc_clean)

0    saya berangkat ke kampus untuk belajar
1             ibu pergi berbelanja ke pasar
2      ayah bekerja di kantor untuk bekerja
Name: clean_sw, dtype: object


In [40]:
#TF-IDF
documents=doc_clean
tokenized_documents = [word_tokenize(doc.lower()) for doc in documents]
preprocessed_documents = [' '.join(doc) for doc in tokenized_documents]
#tokenized_documents
documents[0]
documents[1]
documents[2]
# Convert each document to lowercase and split it into words
tokens1 = documents[0].lower().split()
tokens2 = documents[1].lower().split()
tokens3 = documents[2].lower().split()
# Combine the tokens into a list of unique terms
terms = list(set(tokens1 + tokens2))
print(terms)


['saya', 'kampus', 'ibu', 'pasar', 'belajar', 'ke', 'pergi', 'berangkat', 'berbelanja', 'untuk']


In [41]:
df = pd.DataFrame({"text" :doc_clean})
df

,text
0,saya berangkat ke kampus untuk belajar
1,ibu pergi berbelanja ke pasar
2,ayah bekerja di kantor untuk bekerja


In [42]:
vectorizer = CountVectorizer()

matrix = vectorizer.fit_transform(df['text'])
matrix

<3x14 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [43]:
import pandas as pd

counts = pd.DataFrame(matrix.toarray(),
                      columns=vectorizer.get_feature_names_out())
counts

,ayah,bekerja,belajar,berangkat,berbelanja,di,ibu,kampus,kantor,ke,pasar,pergi,saya,untuk
0,0,0,1,1,0,0,0,1,0,1,0,0,1,1
1,0,0,0,0,1,0,1,0,0,1,1,1,0,0
2,1,2,0,0,0,1,0,0,1,0,0,0,0,1
